In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls=['https://www.victoriaonmove.com.au/index.html','https://www.tataamantra.com/']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [3]:
#data

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#Split Data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

#print(docs)
#print(docs[2])
print("Total number of documents:",len(docs))

Total number of documents: 11


In [7]:
docs[5]

Document(metadata={'source': 'https://www.tataamantra.com/'}, page_content='Price\n\nType Carpet Area Price 2 BHK 617 Sq.Ft ₹0 Price Breakup 3 BHK 852 Sq.Ft ₹0 Price Breakup\n\nENQUIRE NOW\n\nComplete Costing Details\n\nFloor Plan\n\n2 BHK Type 1\n\nENQUIRE NOW\n\n2 BHK Type 2\n\nENQUIRE NOW\n\n2 BHK Type 3\n\nENQUIRE NOW\n\n3 BHK\n\nENQUIRE NOW\n\nGallery\n\nPrevious Next\n\nAmenities\n\nSwimming Pool\n\nGymnasium\n\nLibrary\n\nIndoor Games\n\nMulti Purpose Hall\n\nChildren Play Area\n\nGreen Lawn\n\nSquash Court\n\nMeditation\n\nSenior Citizen Area\n\nDigital Fitness Workout Zone\n\nVirtual Gaming Room\n\nBox Office\n\nMap View\n\nTata Amantra location is at the heart of all major hubs and conveniences. You will find the best of healthcare, education, entertainment. The residences integrate well with the surroundings, with select homes opening to expansive views of the Ulhas River, sunrise, pastures, the skyline of Thane and the neighbouring Kalyan and Thane suburbs.\n\nMumbai-Nashik

In [9]:
from langchain_chroma import Chroma
#from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector =embeddings.embed_query("Hello Nilesh")
vector[:5]
#vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

[-0.004344039596617222,
 0.02825145050883293,
 -0.01968097873032093,
 -0.0031384320463985205,
 0.055905234068632126]

In [10]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [11]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})
retrieved_docs=retriever.invoke("What is tataamantra")
#llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0,max_tokens=None,timeout=None)

In [12]:
retrieved_docs

[Document(id='1264e6a0-26c5-449e-963c-d169ef18ff48', metadata={'source': 'https://www.tataamantra.com/'}, page_content='logo\n\nBOOKING OPEN\n\nTATA Amantra\n\nAt Bhiwandi, Thane By Tata Group\n\nLand Area : Ready To Move In Homes\n\nOC Recieved\n\nLiving Experience That Moves Beyond Just Four Walls\n\nBuy A 3 BHK At The Same Price Of A 2 BHK\n\nReady To Move In Flats\n\nRs. 9000/- Per Sq.Ft. | Offer Valid Till Tomorrow | Hurry Now!\n\n2 & 3 BHK Luxury Homes\n\n₹ 58.5 Lacs* Onwards\n\nFree Cab Facility\n\nFree cab facility for site visit\n\nAt Bhiwandi, Thane\n\nReady to Move In Home at TATA Amantra Offers 2 & 3 BHK Flat In Bhiwandi At 58.5 Lac* Views of Ulhas River, Sunrise. Get Price Benefits, 25000 sq.ft. Clubhouse. Mid and High-rise Towers Of 34 Storeys. Unlock Happiness Ready to Move In Homes at Tata Amantra. Views of Ulhas River, Sunrise. Wake Up to Beautiful View Everyday At Amantra, Kalyan. 25+ Premium Amenities. 20 Acres Land. Bhiwandi-Kalyan Corridor. Grab Booking Offer. Clic

In [13]:
len(retrieved_docs)

10

In [15]:
print(retrieved_docs[5].page_content)

2024 © Copyright - All Rights Reserved. Privacy Policy || Terms & Conditions || Contact Us

Pre-Register here for Best Offers

Free Cab facility

Free cab facility for site visit

Mail Me Pricing Details

Pre-Register here for Best offers

Free Cab

Free cab facility for site visit


In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3,max_tokens=500,timeout=None)


In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [22]:
response = rag_chain.invoke({"input": "What is tataamantra"})
print(response["answer"])

Tata Amantra is a residential project by Tata Housing in Bhiwandi, Thane, India.  It offers 2 & 3 BHK luxury apartments with amenities like a clubhouse, swimming pool, and gym.  The project boasts views of the Ulhas River and is located near the Mumbai-Nashik expressway.


In [24]:
response = rag_chain.invoke({"input": "What is Victoria"})
print(response["answer"])

Victoria On Move is a furniture removal company specializing in furniture removals in Victoria.  They provide various truck sizes and moving services, and they are insured for transit and public liability.  They also offer interstate removal services.
